# To Balance Or Not To Balance: Testing ReLoBRalo
#### Author: JP Melo

In this file we asses training performance using loss balancing methods. In particular, we test the ideas showns in [this article](https://arxiv.org/abs/2110.09813) (ReLoBRaLo).

### Imports

In [1]:
from derpinns.nn import *
from derpinns.utils import *
from derpinns.trainer import *
import torch
import kfac

/Users/josemelo/Desktop/master/tesis/codes/.conda/lib/python3.11/site-packages/kfac/base_preconditioner.py:15: UserWarning: NVIDIA Apex is not installed or was not installed with --cpp_ext. Falling back to PyTorch flatten and unflatten.
  from kfac.distributed import get_rank


## Parameters

In [2]:
# Fix seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Global parameters
assets = 2

sampler = "Sobol"               
nn_shape = "64x3"               
device = torch.device("cpu") 
dtype = torch.float32

# Define option valuation params
params = OptionParameters(
    n_assets=assets,
    tau=1.0,
    sigma=np.array([0.2] * assets),
    rho=np.eye(assets) + 0.25 * (np.ones((assets, assets)) - np.eye(assets)),
    r=0.05,
    strike=100,
    payoff=payoff
)

# Create dataset to traing over
batch_size = 500
total_iter = 1_000
boundary_samples = 20_000
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device)

## Training

We train the same model arquitecture with and without loss balancing. This technique is implemented inside the closure ```LossBalancingDimlessBS```.

### With Loss Balancing

In [3]:
# Build the net to be used
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()

closure = LossBalancingDimlessBS(alpha=torch.tensor(0.90),tau=torch.tensor(1), rho_prob=0.99)\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

Adam training: 100%|██████████| 1000/1000 [02:48<00:00,  5.94it/s, Interior=0.000798, Boundary=0.000657, Initial=0.006812, Total=0.008267, Max Error=94.4718627930, L2 Error=0.1490557194]


In [4]:
with_lb_state = trainer.closure.get_state()
plot_loss(with_lb_state, smooth=True, smooth_window=10)

with_lb_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_lb_results*100)

L2 Error:  4.7739134


## Without Loss Balancing

In [5]:
# Build the net to be used
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

Adam training: 100%|██████████| 1000/1000 [02:50<00:00,  5.86it/s, Interior=0.000759, Boundary=0.000654, Initial=0.005610, Total=0.007022, Max Error=85.5677490234, L2 Error=0.1341786534]


In [6]:
without_lb_state = trainer.closure.get_state()
plot_loss(without_lb_state, smooth=True, smooth_window=10)

without_lb_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", without_lb_results*100)

L2 Error:  3.9504097


6.2328877/3.9504097

### Compare both runs

In [7]:
compare_loss_histories(
    [with_lb_state,without_lb_state],
    ["With Loss Balancing", "Without Loss Balancing"],
    smooth=True,
    smooth_window=50,
)

Comments: No significant improvements can be seen in this particular use case.